In [6]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import folium
import random
import os
import json
import warnings
import time
import os
import csv
import time
import requests

warnings.filterwarnings("ignore")


path_de_redes = 'redes'
path_de_resultados = 'resultados'

os.makedirs(path_de_redes, exist_ok=True)
os.makedirs(path_de_resultados, exist_ok=True)



## Coordenadas geograficas 

In [ ]:
import os
import csv
import time
import requests
import re  # <- añadido

# ==== CONFIGURA AQUÍ ====
INPUT_DIR   = 'Instancias_Equipo4'  # carpeta de entrada
OUTPUT_DIR  = 'redes'               # carpeta de salida
API_KEY     = "apikey"         # o usa variable de entorno GOOGLE_MAPS_API_KEY
PAUSA_SEG   = 0.1                   # pausa corta entre llamadas
PAIS        = "México"
IDIOMA      = "es"
USAR_DEBUG  = True  # si True, agrega columnas geo_status y direccion_consultada
# ========================

def limpiar(x):
    return (str(x).strip()) if x is not None else ""

def construir_direccion(row):
    """
    Crea una dirección tipo:
    "calle num, colonia, C.P., Ciudad, País"
    """
    calle = limpiar(row.get("calle"))
    num   = limpiar(row.get("No.Casa"))
    if calle and num:
        calle = f"{calle} {num}"

    partes = [
        calle or "",
        limpiar(row.get("colonia")),
        limpiar(row.get("C.P.")),
        limpiar(row.get("Ciudad")),
        PAIS
    ]
    return ", ".join([p for p in partes if p])

def geocodificar(direccion, api_key, row=None):
    """
    Llama a Geocoding API con sesgo a MX y components (CP y Ciudad).
    Hace 1 reintento simple si NO hay resultados intentando sin número exterior.
    """
    url = "https://maps.googleapis.com/maps/api/geocode/json"

    # components (mejora precisión en MX)
    cp = limpiar((row or {}).get("C.P."))
    ciudad = limpiar((row or {}).get("Ciudad"))
    comps = ["country:MX"]
    if cp:
        comps.append(f"postal_code:{cp}")
    if ciudad:
        comps.append(f"locality:{ciudad}")

    params_base = {
        "key": api_key,
        "region": "mx",
        "language": IDIOMA,
        "components": "|".join(comps)
    }

    # 1) intento con la dirección completa
    params = dict(params_base)
    params["address"] = direccion
    lat, lon, status = _hit_geocode(url, params)
    if status == "OK":
        return lat, lon, status

    # 2) fallback: si no hubo resultados, intenta sin número exterior
    if status in ("ZERO_RESULTS", "NOT_FOUND", "INVALID_REQUEST"):
        direccion_sin_num = _eliminar_numero(direccion)
        if direccion_sin_num != direccion:
            params2 = dict(params_base)
            params2["address"] = direccion_sin_num
            lat2, lon2, status2 = _hit_geocode(url, params2)
            if status2 == "OK":
                return lat2, lon2, status2

    return "", "", status

def _hit_geocode(url, params):
    try:
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        data = r.json()
        status = data.get("status", "UNKNOWN_ERROR")
        if status == "OK" and data.get("results"):
            loc = data["results"][0]["geometry"]["location"]
            return loc.get("lat"), loc.get("lng"), "OK"
        return "", "", status
    except requests.RequestException as e:
        return "", "", f"ERROR_HTTP: {e}"

def _eliminar_numero(direccion):
    """
    Quita un posible número exterior después del nombre de la calle (heurística simple).
    Ej: "Av. Patria 1234, ..." -> "Av. Patria, ..."
    """
    if "," in direccion:
        primera, resto = direccion.split(",", 1)
    else:
        primera, resto = direccion, ""

    trozos = primera.split()
    if len(trozos) >= 2 and any(ch.isdigit() for ch in trozos[-1]):
        # elimina el último token si parece número
        trozos = trozos[:-1]
    primera_sin = " ".join(trozos)

    if resto.strip():
        return f"{primera_sin}, {resto.strip()}"
    return primera_sin

def procesar_archivo(ruta_entrada, ruta_salida):
    with open(ruta_entrada, "r", encoding="utf-8-sig", newline="") as f_in:
        reader = csv.DictReader(f_in)
        requeridos = {"colonia", "calle", "No.Casa", "C.P.", "Ciudad"}
        faltantes = [h for h in requeridos if h not in reader.fieldnames]
        if faltantes:
            raise RuntimeError(f"Faltan columnas en el CSV: {faltantes}. Encabezados: {reader.fieldnames}")

        fieldnames = reader.fieldnames + ["lat", "lon"]
        if USAR_DEBUG:
            fieldnames += ["geo_status", "direccion_consultada"]

        with open(ruta_salida, "w", encoding="utf-8", newline="") as f_out:
            writer = csv.DictWriter(f_out, fieldnames=fieldnames)
            writer.writeheader()

            cache = {}
            for row in reader:
                direccion = construir_direccion(row)

                if direccion in cache:
                    lat, lon, status = cache[direccion]
                else:
                    lat, lon, status = geocodificar(direccion, API_KEY, row)
                    cache[direccion] = (lat, lon, status)
                    time.sleep(PAUSA_SEG)

                out_row = dict(row)
                out_row["lat"] = lat
                out_row["lon"] = lon
                if USAR_DEBUG:
                    out_row["geo_status"] = status
                    out_row["direccion_consultada"] = direccion

                writer.writerow(out_row)

def main():
    if not API_KEY or API_KEY in ("TU_API_KEY_AQUI", "tu api aqui"):
        raise RuntimeError("Falta API_KEY. Exporta GOOGLE_MAPS_API_KEY o edítala en el script.")

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Recorre todos los CSV de la carpeta de instancias
    archivos = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(".csv")]
    if not archivos:
        print(f"No se encontraron CSV en: {INPUT_DIR}")
        return

    for fname in sorted(archivos):
        ruta_entrada = os.path.join(INPUT_DIR, fname)

        # TSP_E4_<TAM>_<INST>.csv  ->  <INST>_<TAM>.csv
        m = re.match(r'^TSP_E4_(\d+)_(\d+)\.csv$', fname, re.IGNORECASE)
        if m:
            tam, inst = m.group(1), m.group(2)
            nombre_salida = f"{inst}_{tam}.csv"
        else:
            # respaldo por si el patrón no coincide
            base = os.path.splitext(fname)[0]
            nombre_salida = f"{base}.out.csv"

        ruta_salida = os.path.join(OUTPUT_DIR, nombre_salida)

        procesar_archivo(ruta_entrada, ruta_salida)
        print(f"Listo: {fname} -> {nombre_salida}")

if __name__ == "__main__":
    main()


Listo: TSP_E4_100_1.csv -> 1_100.csv
Listo: TSP_E4_100_10.csv -> 10_100.csv
Listo: TSP_E4_100_2.csv -> 2_100.csv
Listo: TSP_E4_100_3.csv -> 3_100.csv
Listo: TSP_E4_100_4.csv -> 4_100.csv
Listo: TSP_E4_100_5.csv -> 5_100.csv
Listo: TSP_E4_100_6.csv -> 6_100.csv
Listo: TSP_E4_100_7.csv -> 7_100.csv
Listo: TSP_E4_100_8.csv -> 8_100.csv
Listo: TSP_E4_100_9.csv -> 9_100.csv
Listo: TSP_E4_150_1.csv -> 1_150.csv
Listo: TSP_E4_150_10.csv -> 10_150.csv
Listo: TSP_E4_150_2.csv -> 2_150.csv
Listo: TSP_E4_150_3.csv -> 3_150.csv
Listo: TSP_E4_150_4.csv -> 4_150.csv
Listo: TSP_E4_150_5.csv -> 5_150.csv
Listo: TSP_E4_150_6.csv -> 6_150.csv
Listo: TSP_E4_150_7.csv -> 7_150.csv
Listo: TSP_E4_150_8.csv -> 8_150.csv
Listo: TSP_E4_150_9.csv -> 9_150.csv
Listo: TSP_E4_200_1.csv -> 1_200.csv
Listo: TSP_E4_200_10.csv -> 10_200.csv
Listo: TSP_E4_200_2.csv -> 2_200.csv
Listo: TSP_E4_200_3.csv -> 3_200.csv
Listo: TSP_E4_200_4.csv -> 4_200.csv
Listo: TSP_E4_200_5.csv -> 5_200.csv
Listo: TSP_E4_200_6.csv -> 6_200

## Aproximacion con cp cuando es necesario 

In [ ]:


# ==== CONFIGURA AQUÍ ====
INPUT_DIR   = 'redes'       # carpeta que ya contiene tus CSV con lat/lon (salida del 1er script)
API_KEY     = "tu api aqui" # o usa variable de entorno GOOGLE_MAPS_API_KEY
PAUSA_SEG   = 0.5           # pausa corta entre llamadas para no saturar la API
IDIOMA      = "es"
# ========================

def limpiar(x):
    return (str(x).strip()) if x is not None else ""

def _hit_geocode(url, params):
    try:
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        data = r.json()
        status = data.get("status", "UNKNOWN_ERROR")
        if status == "OK" and data.get("results"):
            loc = data["results"][0]["geometry"]["location"]
            return loc.get("lat"), loc.get("lng"), "OK"
        return "", "", status
    except requests.RequestException as e:
        return "", "", f"ERROR_HTTP: {e}"

def geocodificar_cp(cp, ciudad=None):
    """
    Obtiene coordenadas aproximadas del centro del C.P. (y opcionalmente ciudad) en MX.
    """
    if not cp:
        return "", "", "SIN_CP"

    url = "https://maps.googleapis.com/maps/api/geocode/json"
    comps = ["country:MX", f"postal_code:{cp}"]
    if ciudad:
        comps.append(f"locality:{ciudad}")

    params = {
        "key": API_KEY,
        "region": "mx",
        "language": IDIOMA,
        "components": "|".join(comps),
        # Con 'address' el geocoder intenta resolver el CP directamente
        "address": f"{cp}, México"
    }
    return _hit_geocode(url, params)

def necesita_aprox(row):
    """
    Decide si una fila necesita aproximación por CP:
    - lat o lon vacíos
    - o geo_status indica fallo típico del 1er script
    """
    lat = limpiar(row.get("lat"))
    lon = limpiar(row.get("lon"))
    if not lat or not lon:
        return True

    status = limpiar(row.get("geo_status"))
    if status and status.upper() in {"ZERO_RESULTS", "NOT_FOUND", "INVALID_REQUEST", "UNKNOWN_ERROR"}:
        return True

    return False

def procesar_archivo(path_csv):
    # Leer completo y reescribir a un temporal
    with open(path_csv, "r", encoding="utf-8", newline="") as f_in:
        reader = csv.DictReader(f_in)
        fieldnames = list(reader.fieldnames)

        # Asegurar la nueva columna al final
        if "aprox_por_cp" not in fieldnames:
            fieldnames.append("aprox_por_cp")

        tmp_path = path_csv + ".tmp"
        with open(tmp_path, "w", encoding="utf-8", newline="") as f_out:
            writer = csv.DictWriter(f_out, fieldnames=fieldnames)
            writer.writeheader()

            cache_cp = {}  # cache (cp|ciudad) -> (lat, lon, status)
            total = 0
            rellenadas = 0

            for row in reader:
                total += 1
                row.setdefault("aprox_por_cp", "0")  # por defecto, no se usó CP

                if necesita_aprox(row):
                    cp = limpiar(row.get("C.P."))
                    ciudad = limpiar(row.get("Ciudad"))

                    clave = (cp, ciudad.lower())
                    if clave in cache_cp:
                        lat_cp, lon_cp, st_cp = cache_cp[clave]
                    else:
                        lat_cp, lon_cp, st_cp = geocodificar_cp(cp, ciudad if ciudad else None)
                        cache_cp[clave] = (lat_cp, lon_cp, st_cp)
                        time.sleep(PAUSA_SEG)

                    if st_cp == "OK" and lat_cp and lon_cp:
                        # Rellenar coordenadas y marcar que se usó CP
                        row["lat"] = lat_cp
                        row["lon"] = lon_cp
                        row["aprox_por_cp"] = "1"
                        rellenadas += 1
                    else:
                        # No se pudo aproximar; mantener como estaba
                        row["aprox_por_cp"] = "0"

                writer.writerow(row)

    # Reemplazar el original
    os.replace(tmp_path, path_csv)
    print(f"Archivo procesado: {os.path.basename(path_csv)} | filas: {total} | rellenadas_por_cp: {rellenadas}")

def main():
    if not API_KEY or API_KEY in ("TU_API_KEY_AQUI", "tu api aqui"):
        raise RuntimeError("Falta API_KEY. Exporta GOOGLE_MAPS_API_KEY o edítala en el script.")

    if not os.path.isdir(INPUT_DIR):
        raise RuntimeError(f"No existe la carpeta: {INPUT_DIR}")

    archivos = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(".csv")]
    if not archivos:
        print(f"No se encontraron CSV en: {INPUT_DIR}")
        return

    for fname in sorted(archivos):
        procesar_archivo(os.path.join(INPUT_DIR, fname))

if __name__ == "__main__":
    main()


## Funciones auxiliares

In [8]:
def haversine_distance(coord1, coord2):
    R = 6371
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = sin(dlat/2)**2 + cos(lat1_rad)*cos(lat2_rad)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def crear_matriz_distancias(df_red):
    coordenadas = df_red[['lat', 'lon']].values
    n = len(coordenadas)
    matriz_distancias = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):
            distancia = haversine_distance(coordenadas[i], coordenadas[j])
            matriz_distancias[i, j] = distancia
            matriz_distancias[j, i] = distancia

    return matriz_distancias

## Algoritmo genético


In [9]:
def inicializar_poblacion(tamano_poblacion, indices_ciudades):
    poblacion = []
    for _ in range(tamano_poblacion):
        individuo = indices_ciudades.copy()
        random.shuffle(individuo)
        poblacion.append(individuo)
    return poblacion

def calcular_aptitud(individuo, matriz_distancias):
    distancia_total = 0
    for i in range(len(individuo)):
        ciudad_origen = individuo[i]
        ciudad_destino = individuo[(i + 1) % len(individuo)]
        distancia_total += matriz_distancias[ciudad_origen][ciudad_destino]
    return distancia_total

def seleccionar_padres(poblacion, aptitudes, num_padres):
    padres_indices = np.argsort(aptitudes)[:num_padres]
    padres = [poblacion[i] for i in padres_indices]
    return padres

def cruzar_padres(padre1, padre2):
    tamano = len(padre1)
    inicio, fin = sorted(random.sample(range(tamano), 2))
    hijo = [None]*tamano
    hijo[inicio:fin+1] = padre1[inicio:fin+1]

    pointer = 0
    for i in range(tamano):
        if hijo[i] is None:
            while padre2[pointer] in hijo:
                pointer += 1
            hijo[i] = padre2[pointer]
            pointer += 1
    return hijo

def mutar_individuo(individuo, tasa_mutacion):
    for i in range(len(individuo)):
        if random.random() < tasa_mutacion:
            j = random.randint(0, len(individuo)-1)
            individuo[i], individuo[j] = individuo[j], individuo[i]
    return individuo

def algoritmo_genetico_tsp(matriz_distancias, tamano_poblacion=100, num_generaciones=100, tasa_mutacion=0.01):
    indices_ciudades = list(range(len(matriz_distancias)))
    poblacion = inicializar_poblacion(tamano_poblacion, indices_ciudades)
    mejor_distancia = float('inf')
    mejor_ruta = None

    for generacion in range(num_generaciones):
        aptitudes = [calcular_aptitud(individuo, matriz_distancias) for individuo in poblacion]
        distancia_actual = min(aptitudes)
        if distancia_actual < mejor_distancia:
            mejor_distancia = distancia_actual
            mejor_ruta = poblacion[np.argmin(aptitudes)]

        num_padres = tamano_poblacion // 2
        padres = seleccionar_padres(poblacion, aptitudes, num_padres)

        siguiente_generacion = []
        while len(siguiente_generacion) < tamano_poblacion:
            padre1, padre2 = random.sample(padres, 2)
            hijo = cruzar_padres(padre1, padre2)
            hijo = mutar_individuo(hijo, tasa_mutacion)
            siguiente_generacion.append(hijo)

        poblacion = siguiente_generacion

        if (generacion + 1) % 50 == 0:
            print(f"Generación {generacion + 1}: Mejor distancia = {mejor_distancia:.2f} km")

    return mejor_ruta, mejor_distancia

## Algoritmo colonia de hormigas (ACO)

In [10]:
def calcular_heuristica(costos):
    num_nodos = costos.shape[0]
    heuristica = np.zeros_like(costos)
    for i in range(num_nodos):
        for j in range(num_nodos):
            if costos[i, j] > 0 and not np.isinf(costos[i, j]):
                heuristica[i, j] = 1 / costos[i, j]
            else:
                heuristica[i, j] = 0
    return heuristica

def calcular_probabilidades(feromonas, heuristica, nodo_actual, visitados, alpha, beta):
    num_nodos = len(feromonas)
    probabilidades = np.zeros(num_nodos)
    for j in range(num_nodos):
        if j not in visitados:
            probabilidades[j] = (feromonas[nodo_actual, j] ** alpha) * (heuristica[nodo_actual, j] ** beta)
    suma_probabilidades = np.sum(probabilidades)
    if suma_probabilidades == 0:
        return probabilidades
    return probabilidades / suma_probabilidades

def construir_camino(feromonas, heuristica, costos, alpha, beta):
    num_nodos = costos.shape[0]
    nodo_actual = random.randint(0, num_nodos - 1)
    camino = [nodo_actual]
    costo_total = 0
    visitados = set(camino)

    while len(camino) < num_nodos:
        probabilidades = calcular_probabilidades(feromonas, heuristica, nodo_actual, visitados, alpha, beta)
        if np.sum(probabilidades) == 0:
            nodos_no_visitados = list(set(range(num_nodos)) - visitados)
            siguiente_nodo = random.choice(nodos_no_visitados)
        else:
            siguiente_nodo = random.choices(range(num_nodos), weights=probabilidades, k=1)[0]
        costo_total += costos[nodo_actual, siguiente_nodo]
        camino.append(siguiente_nodo)
        visitados.add(siguiente_nodo)
        nodo_actual = siguiente_nodo

    costo_total += costos[camino[-1], camino[0]]
    camino.append(camino[0])

    return camino, costo_total

def actualizar_feromonas(feromonas, soluciones, rho):
    feromonas *= (1 - rho)
    for camino, costo in soluciones:
        for i in range(len(camino) - 1):
            feromonas[camino[i], camino[i + 1]] += 1 / costo

def ACO(costos, num_hormigas=500, num_iteraciones=100, alpha=1.0, beta=2.0, rho=0.5):
    num_nodos = costos.shape[0]
    mejor_camino = None
    mejor_costo = float('inf')

    feromonas = np.ones((num_nodos, num_nodos)) * 0.1
    heuristica = calcular_heuristica(costos)

    for iteracion in range(num_iteraciones):
        soluciones = []
        for _ in range(num_hormigas):
            camino, costo = construir_camino(feromonas, heuristica, costos, alpha, beta)
            soluciones.append((camino, costo))
            if costo < mejor_costo:
                mejor_camino = camino
                mejor_costo = costo

        actualizar_feromonas(feromonas, soluciones, rho)

        if (iteracion + 1) % 10 == 0:
            print(f"Iteración {iteracion+1}: Mejor costo hasta ahora = {mejor_costo:.2f} km")

    return mejor_camino, mejor_costo

## Procesar las redes

In [11]:
def procesar_redes(archivo):
    df_red = pd.read_csv(archivo)

    matriz_distancias = crear_matriz_distancias(df_red)

    start_genetico = time.time()
    mejor_ruta_genetico, mejor_distancia_genetico = algoritmo_genetico_tsp(matriz_distancias, tamano_poblacion=100, num_generaciones=500, tasa_mutacion=0.01)
    end_genetico = time.time()

    start_aco = time.time()
    mejor_ruta_aco, mejor_distancia_aco = ACO(matriz_distancias, num_hormigas=500, num_iteraciones=100, alpha=1.0, beta=2.0, rho=0.5)
    end_aco = time.time()

    tiempo_genetico = end_genetico - start_genetico
    tiempo_aco = end_aco - start_aco

    return mejor_ruta_genetico, mejor_distancia_genetico, tiempo_genetico, mejor_ruta_aco, mejor_distancia_aco, tiempo_aco

def crear_visualizacion(df_red, mejor_ruta, nombre_archivo):
    coordenadas = df_red[['lat', 'lon']].values
    ruta_coordenadas = coordenadas[mejor_ruta]
    ruta_coordenadas = np.vstack([ruta_coordenadas, ruta_coordenadas[0]])

    centro_lat = np.mean(coordenadas[:, 0])
    centro_lon = np.mean(coordenadas[:, 1])
    mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=12)

    for idx, coord in enumerate(ruta_coordenadas[:-1]):
        folium.Marker(location=coord, popup=f"Parada {idx + 1}").add_to(mapa)

    folium.PolyLine(ruta_coordenadas, color="blue", weight=2.5, opacity=1).add_to(mapa)

    return mapa.save(nombre_archivo)

def generar_archivos_inclusion(archivo):
        df_red = pd.read_csv(archivo)
        matriz_distancias = crear_matriz_distancias(df_red)
        n = len(df_red)

        # Crear el archivo de inclusión
        nombre_archivo_inc = f"{path_de_resultados}/{archivo.split('/')[-1].split('.')[0]}/gams_{archivo.split('/')[-1].split('.')[0]}.inc"
        with open(nombre_archivo_inc, 'w') as f:
            # Escribir encabezado (índices de columnas)
            f.write("    ")  # Espacio para el índice de filas
            for j in range(n):
                f.write(f"{j:<8}")  # Alinear columnas con espacio fijo
            f.write("\n")

            # Escribir filas de la matriz
            for i in range(n):
                f.write(f"{i:<4}")  # Índice de la fila
                for j in range(n):
                    f.write(f"{matriz_distancias[i][j]:<8.1f}")  # Valor de la matriz con 1 decimal
                f.write("\n")

        print(f"Archivo de inclusión generado: {nombre_archivo_inc}")

def procesar_archivo(archivo):
    print(f"Procesando archivo: {archivo}")
    df_red = pd.read_csv(archivo)

    path_de_resultado_base = f"{path_de_resultados}/{archivo.split('/')[-1].split('.')[0]}"
    os.makedirs(path_de_resultado_base, exist_ok=True)

    path_de_resultado_json = f"{path_de_resultado_base}/{archivo.split('/')[-1].split('.')[0]}.json"
    path_de_visualizacion_genetico = f"{path_de_resultado_base}/{archivo.split('/')[-1].split('.')[0]}_genetico.html"
    path_de_visualizacion_aco = f"{path_de_resultado_base}/{archivo.split('/')[-1].split('.')[0]}_aco.html"

    mejor_ruta_genetico, mejor_distancia_genetico, tiempo_genetico, mejor_ruta_aco, mejor_distancia_aco, tiempo_aco = procesar_redes(archivo)

    # Generar resultados en un archivo json
    with open(path_de_resultado_json, "w") as f:
        json.dump({
            "genetico": {
                "ruta": mejor_ruta_genetico,
                "distancia": mejor_distancia_genetico,
                "tiempo": tiempo_genetico
            },
            "aco": {
                "ruta": mejor_ruta_aco,
                "distancia": mejor_distancia_aco,
                "tiempo": tiempo_aco
            }
        }, f)

    # Crear visualización de la mejor ruta encontrada por el algoritmo genético
    crear_visualizacion(df_red, mejor_ruta_genetico, path_de_visualizacion_genetico)
    crear_visualizacion(df_red, mejor_ruta_aco, path_de_visualizacion_aco)
    print(f"Resultados guardados en: {path_de_resultado_base}\n\n")

# Procesar cada archivo de red
paths = [f"{path_de_redes}/{archivo}" for archivo in os.listdir(path_de_redes)]
paths_sorted = sorted(paths, key=lambda x: int(x.split('/')[1].split('_')[0]))

for archivo in paths_sorted:
    procesar_archivo(archivo)
    generar_archivos_inclusion(archivo)

Procesando archivo: redes/1_100.csv
Generación 50: Mejor distancia = 740.97 km
Generación 100: Mejor distancia = 740.97 km
Generación 150: Mejor distancia = 740.97 km
Generación 200: Mejor distancia = 740.97 km
Generación 250: Mejor distancia = 699.74 km
Generación 300: Mejor distancia = 678.62 km
Generación 350: Mejor distancia = 678.62 km
Generación 400: Mejor distancia = 678.62 km
Generación 450: Mejor distancia = 678.62 km
Generación 500: Mejor distancia = 678.62 km
Iteración 10: Mejor costo hasta ahora = 219.93 km
Iteración 20: Mejor costo hasta ahora = 211.13 km
Iteración 30: Mejor costo hasta ahora = 211.13 km
Iteración 40: Mejor costo hasta ahora = 209.81 km
Iteración 50: Mejor costo hasta ahora = 209.81 km
Iteración 60: Mejor costo hasta ahora = 209.81 km
Iteración 70: Mejor costo hasta ahora = 209.81 km
Iteración 80: Mejor costo hasta ahora = 209.81 km
Iteración 90: Mejor costo hasta ahora = 209.81 km
Iteración 100: Mejor costo hasta ahora = 209.81 km
Resultados guardados en: